In [1]:
%matplotlib inline
import matplotlib.image as mpimg
import numpy as np
import matplotlib.pyplot as plt
# import os,sys
# from PIL import Image

from helpers import *
from sklearn import linear_model
from sklearn.metrics import confusion_matrix

%load_ext autoreload
%autoreload 2

In [2]:
try: 
    import cv2
except: 
    import pip
    pip.main(['install', 'opencv-python'])
    import cv2 

In [25]:
# Loaded a set of images
root_dir = "training/"
image_dir = root_dir + "images/"
gt_dir = root_dir + "groundtruth/"

files = os.listdir(image_dir)

n = len(files)
imgs = [load_image(image_dir + files[i]) for i in range(n)]
gt_imgs = [load_image(gt_dir + files[i]) for i in range(n)]

print("Loading " + str(n) + " satellite + ground truth images")

Loading 100 satellite + ground truth images


In [5]:
# imgs = get_rotated_images(imgs,True)
# gt_imgs = get_rotated_images(gt_imgs,False)

# print("After rotation: " + str(len(imgs)) + " satellite + ground truth images")

In [12]:
# Extract patches from input images
patch_size = 16 # each patch is 16*16 pixels

img_patches = [img_crop_train(imgs[i], patch_size, patch_size) for i in range(n)]
gt_patches = [img_crop_train(gt_imgs[i], patch_size, patch_size) for i in range(n)]

# img_patches = [img_crop(imgs[i], patch_size, patch_size) for i in range(n)]
# gt_patches = [img_crop(gt_imgs[i], patch_size, patch_size) for i in range(n)]


# Linearize list of patches
# img_patches = np.asarray([img_patches[i][j] for i in range(len(img_patches)) for j in range(len(img_patches[i]))])
# gt_patches =  np.asarray([gt_patches[i][j] for i in range(len(gt_patches)) for j in range(len(gt_patches[i]))])


In [13]:
len(img_patches)

100

In [24]:
# Extract features
X = np.asarray([ extract_features(img_patches[i]) for i in range(len(img_patches))])

print("shape X[i] : {}".format(X[0].shape))

# Features augmentation 
X = features_augmentation(X)

# X = normalize(X)

# Print feature statistics
print('Computed ' + str(X.shape[0]) + ' features')
print('Feature dimension = ' + str(X.shape[1]))
# print('Number of classes = ' + str(np.max(Y)))


shape X[i] : (10,)
Computed 62500 features
Feature dimension = 11


In [25]:
def accuracy(labels, predictions):
    """ 
    input:  predictions - prediction array
            labels      - real labels array
    output: acc         - Accuracy: percentage of elements of predictions and labels that are the same
            PPV         - Positive Predictive Value, Precision
            TPR         - True Positive Rate, Sensitivity
    """
    
    conf_mat = confusion_matrix(labels, predictions)

    TN = conf_mat[0,0]
    FP = conf_mat[0,1]
    FN = conf_mat[1,0]
    TP = conf_mat[1,1]
    
    TPR = TP / (TP + FN)
    PPV = TP / (TP + FP)
    
    meanFscore = 2 * PPV * TPR / (PPV + TPR)
    F1 = 0
    
    acc = (TP + TN) / np.sum(conf_mat)
    
#     print("\nTPR = Sensitivity = {}".format(TPR))
#     print("PPV = Precision = {}".format(PPV))
#     print("Mean F Score = {}".format(meanFscore))
#     print("F1 overall = {}".format(F1))
    

    return acc

In [14]:
from sklearn import metrics, cross_validation
from sklearn.model_selection import KFold
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

from plots import cross_validation_visualization

c:\users\prisgdd\anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [40]:
kf = KFold(n_splits = 10)
kf.get_n_splits(imgs)

10

In [44]:
# Compute features for each image patch
# percentage of pixels > 1 required to assign a foreground label to a patch
foreground_threshold = np.arange(0.92, 0.921, 0.01) 
Cs = np.arange(1e4, 1e4 + 1, 10000) 
acc_threshold =[]

accuracy_train_C = []
f1_score_train_C = []

accuracy_test_C = []
f1_score_test_C = []

for C in Cs:
    print("C = {}".format(C))
    for threshold in foreground_threshold:
        print("Foreground threshold = {}".format(threshold))
        def value_to_class(v):
            df = np.sum(v)
            if df > threshold:
                return 1
            else:
                return 0

        Y = np.asarray([value_to_class(np.mean(gt_patches[i])) for i in range(len(gt_patches))])


        accuracy_train_CV = []
        accuracy_test_CV = []
        
        f1_score_train_CV = []
        f1_score_test_CV = []
        
        for ind, [train_index, test_index] in enumerate(kf.split(imgs)):

            ######## Split dataset ########
            print("\n{}-th CV".format(ind+1))
            
            X_train = [imgs[ind] for ind in train_index]
            X_test = [imgs[ind] for ind in test_index]
            
            y_train = [gt_imgs[ind] for ind in train_index]
            y_test = [gt_imgs[ind] for ind in test_index]
            
#             # print("TRAIN:", len(train_index), "TEST:", len(test_index))
#             X_train, X_test = X[train_index], X[test_index]
#             y_train, y_test = y[train_index], y[test_index]
            
            X_train = [img_crop_train(X_train[i], patch_size, patch_size) for i in range(len(train_index))]
            y_train = [img_crop_train(y_train[i], patch_size, patch_size) for i in range(len(train_index))]
            X_test = [img_crop(X_test[i], patch_size, patch_size) for i in range(len(test_index))]
            y_test = [img_crop(y_test[i], patch_size, patch_size) for i in range(len(test_index))]
    
            X_train = np.asarray([X_train[i][j] for i in range(len(X_train)) for j in range(len(X_train[i]))])
            X_test = np.asarray([X_test[i][j] for i in range(len(X_test)) for j in range(len(X_test[i]))])
            y_train = np.asarray([y_train[i][j] for i in range(len(y_train)) for j in range(len(y_train[i]))])
            y_test = np.asarray([y_test[i][j] for i in range(len(y_test)) for j in range(len(y_test[i]))])
        
            y_train = np.asarray([value_to_class(np.mean(y_train[i])) for i in range(y_train.shape[0])])
            y_test = np.asarray([value_to_class(np.mean(y_test[i])) for i in range(y_test.shape[0])])
            
            X_train = np.asarray([ extract_features(X_train[i]) for i in range(len(X_train))])
            X_train = features_augmentation(X_train)
            
            X_test = np.asarray([ extract_features(X_test[i]) for i in range(len(X_test))])
            X_test = features_augmentation(X_test)
            
            
            
            ######## Run logistic regression ########
            print("Logistic Regression // Logistic Regression + Postprocessing")
            logreg = linear_model.LogisticRegression(C=C, class_weight="balanced")
            logreg.fit(X_train, y_train)
            z_train = logreg.predict(X_train)
            z_test = logreg.predict(X_test)

            f1_score_train = f1_score(y_train, z_train, average='macro')
            accuracy_score_train = accuracy_score(y_train, z_train)
            
            f1_score_test = f1_score(y_test, z_test, average='macro')
            accuracy_score_test = accuracy_score(y_test, z_test)

            ######## Postprocessing ########
            # Reshape prediction
            z_reshaped = []

            num_patch_total = len(z_test)
            num_patch_by_img = num_patch_total // kf.get_n_splits(imgs)

            for i in range(0, num_patch_total, num_patch_by_img):
                z_crt = z_test[i : i + num_patch_by_img]
                z_reshaped.append(np.reshape(z_crt, [400 // 16, 400 // 16]))


            # Run post process 
            for ind, label_img in enumerate(z_reshaped):
                label_img = postprocess(label_img)
                z_reshaped[ind] = np.reshape(label_img, [z_crt.shape[0]])

            # Convert list as array
            z_test_pp = np.concatenate( z_reshaped , axis = 0 )

            f1_score_test_pp = f1_score(y_test, z_test_pp, average='macro')
            accuracy_score_test_pp = accuracy_score(y_test, z_test_pp)
#             print(" - - - F1 score test :: LogReg = {} ::: LogReg + PP = {}".format(f1_score_test, f1_score_test_pp))
#             print(" - - - Accuracy score test :: LogReg {} ::: LogReg + PP {}".format(accuracy_score_test, accuracy_score_test_pp))

            
            f1_score_train_CV.append(f1_score_train)
            accuracy_train_CV.append(accuracy_score_train)
            
            f1_score_test_CV.append(f1_score_test_pp)
            accuracy_test_CV.append(accuracy_score_test_pp)
    
    print("Average test accuracy: {}".format(np.mean(accuracy_test_CV)))
    print("Variance test accuracy: {}".format(np.std(accuracy_test_CV)))
    print("Min test accuracy: {}".format(np.min(accuracy_test_CV)))
    print("Max test accuracy: {}\n".format(np.max(accuracy_test_CV)))   
    
    accuracy_train_C.append(np.mean(accuracy_train_CV))
    f1_score_train_C.append(np.mean(f1_score_train_CV))
    
    accuracy_test_C.append(np.mean(accuracy_test_CV))
    f1_score_test_C.append(np.mean(f1_score_test_CV))
        
        
# cross_validation_visualization(Cs, f1_score_train_C, f1_score_test_C)
#     # we create an instance of the classifier and fit the data
#     logreg = linear_model.LogisticRegression(C=1e5, class_weight="balanced")
#     logreg.fit(X, Y)
    
#     # Predict on the training set
#     Z = logreg.predict(X)
    
#     acc = accuracy(labels = Y, predictions = Z)
#     acc_threshold.append(acc)
#     print("Foreground threshold = {}".format(threshold))
#     print("Accuracy post Logistic Regression: {}".format(acc))


C = 10000.0
Foreground threshold = 0.92

1-th CV
Logistic Regression // Logistic Regression + Postprocessing

2-th CV
Logistic Regression // Logistic Regression + Postprocessing

3-th CV
Logistic Regression // Logistic Regression + Postprocessing

4-th CV
Logistic Regression // Logistic Regression + Postprocessing

5-th CV
Logistic Regression // Logistic Regression + Postprocessing

6-th CV
Logistic Regression // Logistic Regression + Postprocessing

7-th CV
Logistic Regression // Logistic Regression + Postprocessing

8-th CV
Logistic Regression // Logistic Regression + Postprocessing

9-th CV
Logistic Regression // Logistic Regression + Postprocessing

10-th CV
Logistic Regression // Logistic Regression + Postprocessing
Average test accuracy: 0.6455200000000001
Variance test accuracy: 0.04209868786553804
Min test accuracy: 0.58112
Max test accuracy: 0.7096



In [ ]:
# Data to evaluate
root_testdir = "test_set_images"
test_names = os.listdir(root_testdir)

num_test = len(test_names)

imgs_test = [load_image(os.path.join(root_testdir, test_names[i], test_names[i]) + ".png") for i in range(num_test)]

img_patches_test = [img_crop(imgs_test[i], patch_size, patch_size) for i in range(num_test)]

# Linearize list of patches
img_patches_test = np.asarray([img_patches_test[i][j] for i in range(len(img_patches_test)) for j in range(len(img_patches_test[i]))])


X_test = np.asarray([ extract_features(img_patches_test[i]) for i in range(len(img_patches_test))])
X_test = features_augmentation(X_test)

# X_test = normalize(X_test)

# Run prediction
Z_test = logreg.predict(X_test)


In [ ]:
""" Postprocessing """

# Reshape prediction
Z_reshaped = []

num_patch_total = len(Z_test)
num_patch_by_img = num_patch_total // num_test

for i in range(0, num_patch_total, num_patch_by_img):
    Z_crt = Z_test[i : i + num_patch_by_img]
    Z_reshaped.append(np.reshape(Z_crt, [608 // 16, 608 // 16]))

    
# Run post process 
for ind, label_img in enumerate(Z_reshaped):
    label_img = postprocess(label_img)
    Z_reshaped[ind] = np.reshape(label_img, [Z_crt.shape[0]])
    
# Convert list as array
result = np.concatenate( Z_reshaped , axis = 0 )

# Save prediction
create_submission(result, "submission_postprocess.csv")

In [ ]:
import random 
patch_size = 16

# Run prediction on the img_idx-th image
img_idx = random.randint(0,n-1)
img_idx = 1

patch_size = 16
Xi = extract_img_features(image_dir + files[img_idx])
Xi = features_augmentation(Xi)
Zi = logreg.predict(Xi)

# Display prediction as an image
w = gt_imgs[img_idx].shape[0]
h = gt_imgs[img_idx].shape[1]

predicted_im = label_to_img(w, h, patch_size, patch_size, Zi)
cimg = concatenate_images(imgs[img_idx], predicted_im)
fig1 = plt.figure(figsize=(10, 10)) # create a figure with the default size 
plt.imshow(cimg, cmap='Greys_r')

new_img = make_img_overlay(imgs[img_idx], predicted_im)

plt.imshow(new_img)

In [ ]:
Zi_reshaped = np.reshape(Zi, [400 // 16, 400 // 16])
postprocess_img = postprocess(Zi_reshaped)
postprocess_img = np.reshape(postprocess_img, [Zi.shape[0]])
postprocess_img = label_to_img(w, h, patch_size, patch_size, postprocess_img)
    
fig1 = plt.figure(figsize=(10, 10)) 
new_img = make_img_overlay(imgs[img_idx], postprocess_img)
plt.imshow(new_img)

In [ ]:
Zi.shape

In [ ]:
Zi_reshaped.shape